In [1]:
"""
Volatility & Value at Risk (VaR) – quick tour
─────────────────────────────────────────────
• Volatility (σ) …… standard deviation of returns; annualise by √252.
• VaR_α ……………… worst loss not exceeded with probability α.
                Here: 1-day, 95 % VaR (α = 0.95).

  ▸ Parametric (Normal):   VaR = z_(α) · σ_1d · S_t
  ▸ Historical (HS):       VaR = empirical (1-α) percentile of past ₺-returns.

Swap the simulated “prices” array with your actual price series and
everything else just works.
"""

import numpy as np
from scipy.stats import norm

# -------------------------------------------------
# 1.  Simulate a price series  (Geometric Brownian Motion)
# -------------------------------------------------
S0, r, sigma_true = 100, 0.02, 0.25
years, steps_per_year = 2, 252
N = years * steps_per_year
dt = 1 / steps_per_year

rng = np.random.default_rng(seed=0)
z   = rng.standard_normal(N)
log_ret_path = (r - 0.5 * sigma_true**2) * dt + sigma_true * np.sqrt(dt) * z
prices = S0 * np.exp(np.cumsum(log_ret_path))      # length N

# -------------------------------------------------
# 2.  Daily log-returns
# -------------------------------------------------
rets = np.diff(np.log(prices))                     # length N-1

# -------------------------------------------------
# 3.  Realised volatility
# -------------------------------------------------
vol_1d = rets.std(ddof=1)                          # σ_daily
vol_annual = vol_1d * np.sqrt(252)                 # σ_annual

# -------------------------------------------------
# 4.  1-day 95 % VaR
# -------------------------------------------------
alpha = 0.95
z_alpha = norm.ppf(alpha)                          # ≈ 1.645
S_t = prices[-1]

# 4a. Parametric (Normal) VaR  – positive number = worst-case loss
VaR_param = z_alpha * vol_1d * S_t

# 4b. Historical-simulation VaR
p_loss = (reit := rets * S_t)                      # $ loss per $ notional
VaR_hist = -np.percentile(p_loss,  (1 - alpha) * 100)

# -------------------------------------------------
# 5.  Report
# -------------------------------------------------
print(f"Sample length               : {N-1} days")
print(f"Realised σ (daily)          : {vol_1d: .4%}")
print(f"Realised σ (annual)         : {vol_annual: .4%}")
print(f"Parametric 95% 1-day VaR    : ${VaR_param:,.2f}")
print(f"Historical 95% 1-day VaR    : ${VaR_hist:,.2f}")


Sample length               : 503 days
Realised σ (daily)          :  1.6011%
Realised σ (annual)         :  25.4173%
Parametric 95% 1-day VaR    : $2.06
Historical 95% 1-day VaR    : $1.84


In [ ]:
#Volatility gauges how wildly prices swing, while Value at Risk says,
#“with 95 % confidence, your worst one-day loss won’t exceed X dollars.”